## ArcGIS I
## Lab 2
## Mohsen Ahmadkhani

Please note that I am doing Part 2 first and then go through part 1. 

# Part 2


In [1]:
import pandas as pd
import requests
import zipfile
import io
import glob, os, sys
import csv
import requests
import re

elev_url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/geodatabase/'
r = requests.get(elev_url)
con = r.content

dir_list = re.findall('4342-\S+(?=\.zip)', str(con), flags = 0)

dirs = []
for d in dir_list:
    dirs.append(elev_url+d.split('"')[0])
    

# cwd = sys.path[0]
cwd = r"C:\Users\umn-ahmad178\Desktop\arc1lab2"
# elev_url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/geodatabase/4342-12-05.gdb.zip'






In [38]:
for d in dirs:
    r = requests.get(d)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path = cwd) # extract to elevation folder



In [ ]:
import pandas as pd
# import geopandas as gpd
import requests
import zipfile
import io
import glob, os, sys
import csv
import requests
import re
winona = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/winona/elevation_data.gdb.zip'
wabasha = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/wabasha/elevation_data.gdb.zip'

r = requests.get(winona)
# con = r.content
# r = requests.get(d)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path = cwd) # extract to elevation folder


In [14]:
bbox = [[-92.1532542995,44.0388220413],[-92.1540582204,44.1635770527],[-91.9661190296,44.1642009246],[-91.9653151087,44.0394472311],[-92.1532542995,44.0388220413]]


import arcpy

# Create a polygon geometry
array = arcpy.Array([arcpy.Point(bbox[0][0],bbox[0][1]),
                     arcpy.Point(bbox[1][0],bbox[1][1]),
                     arcpy.Point(bbox[2][0],bbox[2][1]),
                     arcpy.Point(bbox[3][0],bbox[3][1])
                     ])

# array = arcpy.Array([arcpy.Point(bbox[0][1],bbox[0][0]),
#                      arcpy.Point(bbox[1][1],bbox[1][0]),
#                      arcpy.Point(bbox[2][1],bbox[2][0]),
#                      arcpy.Point(bbox[3][1],bbox[3][0])
#                      ])
rectangle = arcpy.Polygon(array)
out_raster = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\clipped_elev"






In [2]:
# arcpy.ddd.LasDatasetToTin(in_las_dataset, out_tin, 
#                           {thinning_type}, {thinning_method}, {thinning_value}, {max_nodes},
#                           {z_factor}, {clip_to_extent})

import arcpy
dem_3m = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\dem_3m_m'
p = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb'

arcpy.env.workspace = p  # setting workspace to the current directory
arcpy.env.overwriteOutput = True




In [9]:
arcpy.env.workspace

'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\elevation_data.gdb'

In [10]:
arcpy.ListRasters()

['dem_1m_m', 'dem_3m_m', 'dem_3m_hs', 'dem_1m_hs']

In [31]:
#clip to the counties

winona_dem = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\dem_3m_m'
wabasha_dem = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\wabasha_dem.gdb\dem_3m_m'
olmsted_dem = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\olmsted_dem.gdb\dem_3m_m'


rast_olmsted = arcpy.Raster(olmsted_dem)
rast_winona = arcpy.Raster(winona_dem)
rast_wabasha = arcpy.Raster(wabasha_dem)
rec = '564711 4890814 585764 4873865'

olmsted_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\olmsted_dem_3m_clip_out'
winona_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\winona_dem_3m_clip_out'
wabasha_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\wabasha_dem_3m_clip_out'

arcpy.management.Clip(rast_olmsted, rec, olmsted_out)
arcpy.management.Clip(rast_winona, rec, winona_out)
arcpy.management.Clip(rast_wabasha, rec, wabasha_out)


<Result 'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\elevation_data.gdb\\wabasha_dem_3m_clip_out'>

In [32]:
import arcpy
from arcpy.ia import *

in_raster1 = arcpy.Raster(olmsted_out)
in_raster2 = arcpy.Raster(winona_out)
in_raster3 = arcpy.Raster(wabasha_out)

# Merge the three rasters
bbox_rast = arcpy.ia.Merge([in_raster1, in_raster2, in_raster3], "FIRST")
bbox_rast.save(r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\merged_rast")

In [19]:
# bounding box
arcpy.conversion.FeatureClassToShapefile(rectangle, r'C:\Users\umn-ahmad178\Desktop\arc1lab2')




<Result 'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2'>

In [33]:
#Create slope
from arcpy.sa import *

# Set local variables
inRaster = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\merged_rast"
outMeasurement = "DEGREE"
# zFactor = 0.3043

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

# Execute Slope
outSlope = Slope(inRaster, outMeasurement)

# Save the output 
outSlope.save(r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\slope_rast")

In [35]:
# download the landcover data 
farm_tif_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/tif_base_landcover_minnesota.zip'

r = requests.get(farm_tif_url)

z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path = r"C:\Users\umn-ahmad178\Desktop\arc1lab2") # extract to elevation folder




In [37]:
# clipping the raster file to the bbox
landcover_tif_path = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\landcover_impervious_statewide2013_v2.tif'

landcover_tif = arcpy.Raster(landcover_tif_path)
rec = '564711 4890814 585764 4873865'


landcover_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_out'

arcpy.management.Clip(landcover_tif, rec, landcover_out)




<Result 'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\elevation_data.gdb\\landcover_out'>

In [44]:
# Resample TIFF image to a higher resolution
    

landcover = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_reclass'
slope = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\slope_rast"
elev = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\merged_rast"

landcover_out = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_3m"
slope_out = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\slope_3m"
elev_out = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\dem_3m"

arcpy.management.Resample(landcover, landcover_out, 3, "CUBIC")
arcpy.management.Resample(slope, slope_out, 3, "CUBIC")
arcpy.management.Resample(elev, elev_out, 3, "CUBIC")


<Result 'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\elevation_data.gdb\\dem_3m'>

In [46]:

import arcpy
# Reclassified according to:
# https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/metadata/metadata.html

# Set local variables
inRaster = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_out'
field = "VALUE"
remapString = "0 100 0;101 102 1;103 256 0"
outRaster = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\water_bodies'

# Execute Reclassify
arcpy.Reclassify_3d(inRaster, field, remapString, outRaster, "DATA")


# Set local variables
inRaster = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_out'
field = "VALUE"
remapString = "0 108 0;109 110 1;111 256 0"
outRaster = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\farm'

# Execute Reclassify
arcpy.Reclassify_3d(inRaster, field, remapString, outRaster, "DATA")



<Result 'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\elevation_data.gdb\\farm'>

In [10]:
# Raster Calculator ----------> Cost map creation
import arcpy
from arcpy.sa import *
import arcpy
dem_3m = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\dem_3m_m'
p = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb'

arcpy.env.workspace = p  # setting workspace to the current directory
arcpy.env.overwriteOutput = True
# # Check out the ArcGIS Spatial Analyst extension license
# arcpy.CheckOutExtension("Spatial")


# Set local variables
slope = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\slope_3m"
farm = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\farm'
waterbodies = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\water_bodies'
# Excuate RasterCalculator(Minus) function
cost_s_m1_f_m10_w_m1 = RasterCalculator([slope, farm, waterbodies], ["s", "f", "w"],
                                       "(-1)*s+(-10)*f+(-1)*w")
cost_s_m1_f_m10_w_m1_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1'
# Save the output
cost_s_m1_f_m10_w_m1.save(cost_s_m1_f_m10_w_m1_out)

In [11]:


ras = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1'
cost_s_m1_f_m10_w_m1 = RasterCalculator([ras], ["p"],
                                       "255-255*((p + 59.497318267822266) / 59.497318267822266)")
cost_s_m1_f_m10_w_m1_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1_normal_reversed'
# Save the output
cost_s_m1_f_m10_w_m1.save(cost_s_m1_f_m10_w_m1_out)


In [ ]:
#create farm raster

landcover_in = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_out'
field = "VALUE"
remapString = "0 100 0;101 102 1;103 103 2;104 104 3;105 107 4;108 108 5;109 110 6"
outRaster = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\landcover_reclass'

# Execute Reclassify
arcpy.Reclassify_3d(inRaster, field, remapString, outRaster, "DATA")

In [17]:
od = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\OD.shp'
out_path = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\out_path_rev'
# cost_s_m1_f_m10_w_m1_normal = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1_normal'
cost_s_m1_f_m10_w_m1_normal_reversed = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1_normal_reversed'
result = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\result_normal.shp'

# OptimalRegionConnections(od, out_path, {in_barrier_data}, 
#                          {in_cost_raster = }, {out_neighbor_paths}, {distance_method}, {connections_within_regions})

OptimalRegionConnections(od, out_path, in_cost_raster =cost_s_m1_f_m10_w_m1_normal_reversed) #correct one




<geoprocessing server result object at 0x25651984030>

In [ ]:
cost_s_m1_f_m10_w_m1_normal_reversed = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1_normal_reversed'
out_path_rev = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\out_path_rev'
# landcover_tif_path = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\landcover_impervious_statewide2013_v2.tif'

landcover_tif = arcpy.Raster(cost_s_m1_f_m10_w_m1_normal_reversed)
# arcpy.Polyline()
# landcover_tif


In [24]:
# Raster Calculator ----------> Cost map creation
import arcpy
from arcpy.sa import *
import arcpy

p = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb'

arcpy.env.workspace = p  # setting workspace to the current directory
arcpy.env.overwriteOutput = True
# # Check out the ArcGIS Spatial Analyst extension license
# arcpy.CheckOutExtension("Spatial")


# Set local variables
slope = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\slope_3m"
farm = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\farm'
waterbodies = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\water_bodies'
# Excuate RasterCalculator(Minus) function
cost_s_m1_f_m4_w_m3 = RasterCalculator([slope, farm, waterbodies], ["s", "f", "w"],
                                       "(-1)*s+(-4)*f+(-3)*w")
cost_s_m1_f_m4_w_m3_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m4_w_m3'
# Save the output
cost_s_m1_f_m4_w_m3.save(cost_s_m1_f_m4_w_m3_out)


In [25]:
#normalize to [0,255]
# ras = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1'
cost_s_m1_f_m4_w_m3_n_r = RasterCalculator([cost_s_m1_f_m4_w_m3_out], ["p"],
                                       "255-255*((p + 59.497318267822266) / 59.497318267822266)")
cost_s_m1_f_m4_w_m3_n_r_out = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m4_w_m3_n_r'
# Save the output
cost_s_m1_f_m4_w_m3_n_r.save(cost_s_m1_f_m4_w_m3_n_r_out)



In [26]:
od = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\OD.shp'
out_path = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m4_w_m3_n_r_out_path'
# cost_s_m1_f_m10_w_m1_normal = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\cost_s_m1_f_m10_w_m1_normal'
# result = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\result_normal.shp'

# OptimalRegionConnections(od, out_path, {in_barrier_data}, 
#                          {in_cost_raster = }, {out_neighbor_paths}, {distance_method}, {connections_within_regions})

OptimalRegionConnections(od, out_path, in_cost_raster =cost_s_m1_f_m4_w_m3_n_r_out) #correct one



<geoprocessing server result object at 0x256647057d8>

# Part 1

*	Downloads .LAS files from MN DNR [1] 
*	Converts the .LAS file into both a DEM and a TIN
*	Saves the new DEM and TIN to disk
*	Exports PDFs of the DEM and TIN with correct visualization (see here)


In [27]:
# Download las files
import pandas as pd
# import geopandas as gpd
import requests
import zipfile
import io
import glob, os, sys
import csv
import requests
import re

elev_url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/'
r = requests.get(elev_url)
con = r.content

dir_list = re.findall('4342-\S+(?=\.las)', str(con), flags = 0)

dirs = []
for d in dir_list:
    dirs.append(elev_url+d.split('"')[0])
    

# cwd = sys.path[0]
cwd = r"C:\Users\umn-ahmad178\Desktop\arc1lab2"
# elev_url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/geodatabase/4342-12-05.gdb.zip'
dirs


['https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-05.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-06.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-07.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-13-03.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-13-04.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-13-05.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-13-06.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-13-07.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-14-03.las',
 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lida

In [55]:
#download all las files
import urllib.request as ur

for url in dirs[0:3]:
    testfile = ur.URLopener()
    testfile.retrieve(url, f'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\las_files\\{url[-14:]}')

    


In [47]:
dirs[0][-14:]

'4342-12-05.las'

In [77]:
# Convert las to DEM

in_las_dataset = f'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\las_files\\{dirs[0][-14:]}'

OUT = r'c:\users\umn-ahmad178\documents\arcgis\projects\arc1lab2\arc1lab2.gdb\c43421205_la2'


arcpy.LasDatasetToRaster_conversion(in_las_dataset, OUT, 'ELEVATION', 'BINNING AVERAGE LINEAR', 'FLOAT', 'CELLSIZE', 10,1)



<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\c43421205_la2'>

In [78]:
# Convert las to tin
OUT = r'c:\users\umn-ahmad178\documents\arcgis\projects\arc1lab2\arc1lab2.gdb\out_tin'

in_las_dataset = f'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\las_files\\{dirs[0][-14:]}'

arcpy.ddd.LasDatasetToTin(in_las_dataset, OUT, 'RANDOM', 15, 3.28)
#                           {thinning_type}, {thinning_method}, {thinning_value}, {max_nodes},
#                           {z_factor}, {clip_to_extent})



<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\out_tin'>

In [97]:
# Export DEM layout to pdf
proj_path = r'C:\Users\umn-ahmad178\Documents\ArcGIS\Projects\arc1lab2\arc1lab2.aprx'
work_dir = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\las_to_dem.pdf"

aprx = arcpy.mp.ArcGISProject(proj_path)
lyt = aprx.listLayouts()[0]
lyt.exportToPDF(work_dir, resolution = 300)


'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\las_to_dem.pdf'

In [98]:
# Export TIN layout to pdf
proj_path = r'C:\Users\umn-ahmad178\Documents\ArcGIS\Projects\arc1lab2\arc1lab2.aprx'
work_dir = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\las_to_tin.pdf"

aprx = arcpy.mp.ArcGISProject(proj_path)
lyt = aprx.listLayouts()[1]
lyt.exportToPDF(work_dir, resolution = 300)


'C:\\Users\\umn-ahmad178\\Desktop\\arc1lab2\\las_to_tin.pdf'

# Part 1.3.
Describe and build an ETL in ArcPro Jupyter Notebooks that
1. Downloads the annual 30-Year Normals .bil files for precipitation from PRISM [2]
*	Converts the data into a spacetime cube and exports it to disk (see here for example of final conversion step; to get to this point, you will need to go through other transformation steps likely) [3]
*	Export an animation of the timeseries


In [103]:
# Download PRISM precipitation data using FTP 
import ftplib

path = '/normals_4km/ppt' 
filename = 'PRISM_ppt_30yr_normal_4kmM2_annual_bil.zip'

ftp = ftplib.FTP("prism.nacse.org") 
ftp.login("anonymous", "ahmad178@umn.edu") 

ftp.cwd(path)
ftp.nlst()
ftp.retrbinary("RETR " + filename, open(r"C:\Users\umn-ahmad178\Desktop\arc1lab2\\"+filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

In [107]:
# Unzip the file 
import zipfile
import os
import glob

bill_file = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\\"+filename
z = zipfile.ZipFile(bill_file)
z.extractall(path = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\bill\\") # extract to bill folder
os.chdir(r"C:\Users\umn-ahmad178\Desktop\arc1lab2\bill\\")
bil = glob.glob("*.bil")[0]
bil

'PRISM_ppt_30yr_normal_4kmM2_annual_bil.bil'

In [ ]:
output_cube = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\out_cube\\"
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(bil, output_cube, 'ZEROS')


In [111]:

path = '/normals_4km/ppt' 
filename = 'PRISM_ppt_30yr_normal_4kmM2_annual_bil.zip'
filename = 'PRISM_ppt_30yr_normal_4kmM2_all_bil.zip'
ftp = ftplib.FTP("prism.nacse.org") 
ftp.login("anonymous", "ahmad178@umn.edu") 

ftp.cwd(path)
ftp.nlst()
ftp.retrbinary("RETR " + filename, open(r"C:\Users\umn-ahmad178\Desktop\arc1lab2\\"+filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

In [115]:
# Unzip the file 
import zipfile
import os
import glob
filename = '\PRISM_ppt_30yr_normal_4kmM2_all_bil.zip'
bill_file = r"C:\Users\umn-ahmad178\Desktop\arc1lab2"+filename
z = zipfile.ZipFile(bill_file)
z.extractall(path = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\bill\\") # extract to bill folder
os.chdir(r"C:\Users\umn-ahmad178\Desktop\arc1lab2\bill\\")
bil = glob.glob("*.bil")
bil

['PRISM_ppt_30yr_normal_4kmM2_01_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_02_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_03_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_04_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_05_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_06_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_07_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_08_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_09_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_10_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_11_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_12_bil.bil',
 'PRISM_ppt_30yr_normal_4kmM2_annual_bil.bil']

In [117]:
# in_workspace = r"C:\Users\umn-ahmad178\Desktop\arc1lab2"
in_workspace = r'c:\users\umn-ahmad178\documents\arcgis\projects\arc1lab2\arc1lab2.gdb'
coordinate_system = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\bill\PRISM_ppt_30yr_normal_4kmM2_04_bil.bil"

arcpy.management.CreateMosaicDataset(in_workspace, 'precipitation', coordinate_system)




<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\precipitation'>

In [119]:
in_mosaic_dataset = r'c:\Users\umn-ahmad178\documents\ArcGIS\Projects\arc1lab2\arc1lab2.gdb\precipitation'
input_path = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\bill"
arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset, 'Raster Dataset', input_path)


<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\precipitation'>

In [155]:
# calculate field (ppt = precipitation)
in_table = r"c:\Users\umn-ahmad178\documents\ArcGIS\Projects\arc1lab2\arc1lab2.gdb\precipitation"

arcpy.AddField_management(in_table, "Variable", "TEXT")
arcpy.CalculateField_management(in_table, "Variable", "'ppt'", "PYTHON3", "")



<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\precipitation'>

In [156]:
# calculate field (ppt = precipitation)
in_table = r"c:\Users\umn-ahmad178\documents\ArcGIS\Projects\arc1lab2\arc1lab2.gdb\precipitation"
expression = "DateAdd(Date(2010,0,1), $feature.OBJECTID-1, 'months')"

expression = "DateAdd(Date(2010,1,1),$feature.OBJECTID-2, 'months')"

arcpy.AddField_management(in_table, "Timestamp", "Date")

arcpy.CalculateField_management(in_table, "Timestamp", expression, "Arcade", "")



<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\precipitation'>

In [157]:
inputmosaicdataset = r"c:\Users\umn-ahmad178\documents\ArcGIS\Projects\arc1lab2\arc1lab2.gdb\precipitation"
variable_field = "Variable"
dimension_fields= "Timestamp"

arcpy.BuildMultidimensionalInfo_md(inputmosaicdataset, variable_field, dimension_fields)




<Result 'c:\\Users\\umn-ahmad178\\documents\\ArcGIS\\Projects\\arc1lab2\\arc1lab2.gdb\\precipitation'>

In [158]:
in_multidimensional_raster = r"c:\Users\umn-ahmad178\documents\ArcGIS\Projects\arc1lab2\arc1lab2.gdb\precipitation"
out_multidimensional_raster_layer = r"precipitation_MultidimLayer"
variables = "ppt"
dimension_def = "ALL"
dimension_values = ""
template = "-2871587.5494 2660354.42205404 3264899.3805797 6041683.9549"

arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster, out_multidimensional_raster_layer, variables, 
                                         dimension_def, dimension_values, template)


<Result 'precipitation_multidimlayer'>

In [ ]:

out_path = r"C:\Users\umn-ahmad178\Desktop\arc1lab2\precipitation_st_cube.nc"
arcpy.CreateSpaceTimeCubeMDRasterLayer_stpm(r"precipitation_MultidimLayer", 
                  out_path, "SPACE_TIME_NEIGHBORS")
